In [1]:
import yaml
import os
import glob
import json
from datetime import datetime

prj = r"Z:\LWI\FIA Darlington\AmiteWatershed_2016Event_WithDarlingtonReservoir\AmiteWatershed_2016Event.prj"
prj_dir, prj_file_tail = os.path.split(prj)
prj_name = prj_file_tail.split(".")[0]

with open(prj, "r") as f:
     lines = f.readlines()
lines = [s.strip('\n') for s in lines]
lines

['Version=3.0',
 'ProjectName=AmiteWatershed_2016Event',
 'Extents=1240630.432,2965050.063,1976174.373,2623870.063',
 'Grow Extents=false',
 'Created=1549043478759',
 'Created By=kslover',
 'UnitSystemID=1',
 'SyncMapWindows=false',
 'DrawMapGridLines=true',
 'MapBackgroundColor=white',
 '',
 'CoordinateInfoBegin',
 'type=X-Y',
 'units=U.S. Feet',
 'unitMult=1.0',
 'spheroid=',
 'CoordinateInfoEnd',
 '',
 '',
 'MapBegin',
 'Name=Amite_StructureInventory_NAD83_Albers.shp',
 'Description=',
 'Path=maps/Amite_StructureInventory_NAD83_Albers.shp',
 'Shown=true',
 'Default=true',
 'Class=hec.map.aishape.AiShapeMap',
 'MaxScaleValue=2147483647',
 'MinScaleValue=-2147483648',
 'GlyphDataRecordSerialized=hec.map.aishape.AiShapeGlyphDataRecord,maps/Amite_StructureInventory_NAD83_Albers.gdr',
 'MapEnd',
 '',
 'MapBegin',
 'Name=AmiteParishes_NAD83_Albers.shp',
 'Description=',
 'Path=maps/AmiteParishes_NAD83_Albers.shp',
 'Shown=true',
 'Default=true',
 'Class=hec.map.aishape.AiShapeMap',
 'MaxS

In [17]:
import copy
keyValueList = copy.deepcopy(lines)  

# get fia version, author, and date
for i,v in enumerate(lines):
        if 'Version=' in v:
                proj_fia_version = v.split('=')[-1]
        if 'Created=' in v:
                proj_created_date_unix = v.split('=')[-1]
        if 'Created By=' in v:
                proj_author = v.split('=')[-1]
        
# Find the first wanted header to start parsing Map and Manager blocks, get the line number, break the loop.
for i,v in enumerate(lines):
        if (v == 'MapBegin') or (v == 'ManagerBegin'):
                startLine_after_proj_info = i
                break

# Parse the wanted blocks of Manager and Map.
nestList = []
for i,v in enumerate(lines):
        if (v == 'ManagerEnd') or (v == 'MapEnd'):
                # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                if len(nestList) > 0:
                        nestList.append(lines[startLine_after_proj_info+1:i])
                else:
                        nestList.append(lines[startLine_after_proj_info:i])
                startLine_after_proj_info = i+1



19


In [22]:
# Break the nestList into 2 lists each for Maps and Managers by chcking the first value in each subList.
# Only appending the values in each subList from columns [1:4]. The others deemed unnecessary.
mapList = []
managerList = []
for subList in nestList:
    if subList[0] == 'MapBegin':
        mapList.append(subList[1:4])
    if subList[0] == 'ManagerBegin':
        managerList.append(subList[1:4])